In [106]:
import matplotlib.pyplot as plt
import os
import sys
import numpy as np
import keras
from keras.applications.inception_v3 import InceptionV3
from keras.applications.vgg19 import VGG19
from keras.preprocessing import image
from keras.models import Model

# Load models

In [111]:
base_vgg_model = VGG19(weights='imagenet', include_top=True)
featrue_extract_model = Model(inputs=base_vgg_model.input, outputs=base_vgg_model.get_layer('flatten').output)

# Preprocess Image : Extract Features

In [117]:
image_size = 224
feature_size = 7 * 7 * 512
batch_size = 100

def load_image(path):
    img = image.load_img(img_path, target_size=(image_size, image_size))
    return image.img_to_array(img)

def load_all_image(path, files):
    length = len(files)
    imgs = np.zeros((length, image_size, image_size, 3))
    labels = np.zeros(length)
    for i, file in enumerate(files):
        imgs[i] = load_image(file)
        labels[i] = 1 if "dog" in file else 0
    return imgs, labels

def get_batchs(files, batch_size):
    length = len(files)
    total_batch = (length + batch_size - 1) // batch_size
    for i in range(total_batch):
        start = i * batch_size
        end = start + batch_size
        if end > length:
            end = length
        yield start, end, files[start: end]

def extract_features(folder, max_size=-1):
    features = np.zeros((len(train_files), feature_size))
    labels = np.zeros(len(train_files))
    
    all_files = os.listdir(folder)[0:max_size]
    for start, end, files in get_batchs(all_files, batch_size):
        imgs, labels = load_all_image('data/train', files)
        pred = featrue_extract_model.predict(imgs)
        features[start:end] = pred
        labels[start:end] = labels
        print("process batch ", folder, start, end)
    return features, labels

In [118]:
train_features, train_labels = extract_features('data/train', max_size=2)
test_features, test_labels = extract_features('data/test', max_size=2)       
np.save("train_features.npy", train_features)
np.save("train_labels.npy", train_labels)
np.save("test_features.npy", test_features)
np.save("test_labels.npy", test_labels)
print("data saved")

process batch  data/train 0 2
process batch  data/test 0 2
data saved


In [128]:
test_labels

array([ 0.,  0.])

# Check Point

In [157]:
train_features = np.load("train_features.npy")
train_labels = np.load("train_labels.npy")
test_features = np.load("test_features.npy")
test_labels = np.load("test_labels.npy")

In [92]:
img = load_images('data/train/dog.2.jpg')
img = np.expand_dims(img, axis=0)
vgg16_magenet.predict(img)

array([[[[  0.        ,   0.        ,   0.        , ...,   0.        ,
           19.664711  ,   0.        ],
         [  9.37395573,   0.        ,   0.        , ...,   0.        ,
            0.        ,   0.        ],
         [  0.        ,   0.        ,   0.        , ...,   0.        ,
            0.        ,   1.17775691],
         ..., 
         [  0.        ,   0.        ,  20.53322983, ...,   0.        ,
            0.        ,   0.        ],
         [  3.18618083,   0.        ,   7.12070274, ...,   0.        ,
            0.        ,   0.        ],
         [  0.        ,   0.        ,   0.        , ...,   0.        ,
            0.        ,   0.        ]],

        [[  0.        ,   0.        ,   0.        , ...,   0.        ,
           24.4475708 ,   0.        ],
         [ 20.37709808,   0.        ,   2.97575259, ...,   0.        ,
            0.        ,   0.        ],
         [ 16.60351372,   0.        ,   7.79169703, ...,   0.        ,
            0.        ,   0.    

# Train on Image Features

In [187]:
def create_dog_cat_model(inputs, is_training=True, scope="dog_cat_model"):
    with tf.variable_scope(scope, 'dog_cat_model', [inputs]):
        end_points = {}
        
        with slim.arg_scope([slim.fully_connected], activation_fn=tf.nn.relu):

            net = slim.fully_connected(inputs, 1024, scope='fc1')
            end_points['fc1'] = net

            net = slim.dropout(net, 0.8, is_training=is_training)

            net = slim.fully_connected(net, 1024, scope='fc2')
            end_points['fc2'] = net

            net = slim.fully_connected(net, 1, activation_fn=None, scope='logits')
            end_points['logits'] = net

            predictions = tf.nn.softmax(net, name='predictions')
            end_points['predictions'] = tf.nn.sigmoid(logits, name='predictions')
            return predictions, end_points

In [197]:
ckpt_dir = 'dog_cat_model'

dog_cat_graph = tf.Graph()
with dog_cat_graph.as_default():
    tf.logging.set_verbosity(tf.logging.INFO)
    
    inputs = tf.placeholder(tf.float64, shape=(None, 1536), name="inputs")
    labels = tf.placeholder(tf.float64, shape=(None), name="labels")

    predictions, end_points = create_dog_cat_model(inputs, is_training=True)
    loss = tf.losses.log_loss(labels, predictions)

    optimizer = tf.train.AdamOptimizer(learning_rate=0.005)
    train_op = optimizer.minimize(loss)
    init_op = tf.global_variables_initializer()
    with tf.Session() as sess:
        sess.run(init_op)
        
        loss_val,_ = sess.run([loss, train_op], feed_dict={inputs:train_features, labels:train_labels})
        print("Training Loss ", loss_val)

print("Training Finished")

Training Loss  16.1181
Training Finished


In [192]:
train_labels

array([ 0.,  0.])

In [27]:
test1 = np.zeros((10,244,244,3))
test2 = load_image('data/train/dog.l.jpg')
test3 = np.ones((244,2))
test1[1] = test2

ValueError: could not broadcast input array from shape (224,224,3) into shape (244,244,3)

In [33]:
type(test1[1,:])


numpy.ndarray

In [34]:
type(test2)

numpy.ndarray

In [37]:
dir(vgg16_magenet)

['__call__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_add_inbound_node',
 '_built',
 '_feed_input_names',
 '_feed_input_shapes',
 '_feed_inputs',
 '_fit_loop',
 '_get_node_attribute_at_index',
 '_make_predict_function',
 '_make_test_function',
 '_make_train_function',
 '_output_mask_cache',
 '_output_shape_cache',
 '_output_tensor_cache',
 '_per_input_losses',
 '_per_input_updates',
 '_predict_loop',
 '_standardize_user_data',
 '_test_loop',
 '_updated_config',
 'add_loss',
 'add_update',
 'add_weight',
 'assert_input_compatibility',
 'build',
 'built',
 'call',
 'compile',
 'compute_mask',
 'compute_output_shape',
 'constraints',
 'container_nodes',
 'count_p

In [39]:
dir( keras.applications.vgg16 )

['Conv2D',
 'Dense',
 'Flatten',
 'GlobalAveragePooling2D',
 'GlobalMaxPooling2D',
 'Input',
 'K',
 'MaxPooling2D',
 'Model',
 'VGG16',
 'WEIGHTS_PATH',
 'WEIGHTS_PATH_NO_TOP',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__spec__',
 '_obtain_input_shape',
 'absolute_import',
 'decode_predictions',
 'get_file',
 'get_source_inputs',
 'layer_utils',
 'preprocess_input',
 'print_function',
 'warnings']

In [44]:
test2.shape

(224, 224, 3)

In [49]:
test2.reshape((224, -1))

array([[  25.,   13.,   23., ...,  107.,  111.,  110.],
       [  29.,   17.,   27., ...,  110.,  110.,  110.],
       [  22.,   10.,   20., ...,  112.,  110.,  111.],
       ..., 
       [ 201.,   44.,   73., ...,  239.,   68.,   87.],
       [ 197.,   54.,   74., ...,  242.,   63.,   85.],
       [ 189.,   46.,   66., ...,  242.,   63.,   85.]], dtype=float32)

In [54]:
np.ones(10)[0:-1]

array([ 1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.])